This notebook performs simple radiative transfer of Lyman-Birge-Hopfield band emissions along a single atmospheric column. GLOW output fields are required to perform the radiative transfer. The top of atmosphere radiances are then compared with SSUSI observations.

In [59]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset

In [196]:
# Define (approximate) O2 absorption cross section over LBHl and LBHs
x_o2_l = 6.5222e-19 # 140-150 nm mean from o2 cross section file
x_o2_s = 1.27775e-17 # 165-180 nm mean from o2 cross section file

# Define fraction of LBH bandwidth in LBHl and LBHs
frac_lbhs = 0.25 
frac_lbhl = 0.21 

In [197]:
# Read csv GLOW output file
csv_filename = 'F16_2014220_29344.csv'

# Read csv file into pandas dataframe, skipping header rows
data = pd.read_csv(csv_filename,skiprows = 11)

# Grab data needed for radiative transfer calculation and convert to numpy array
rt_data = np.flip(data[['# z','O2','LBH']].to_numpy(),axis=0) # flipped such that higher altitudes are at beginning

In [198]:
# Compute path length for each vertical location
dpath = np.empty(np.size(rt_data[:,0]))
for i in range(np.size(rt_data[:,0])-1):
    dpath[i] = rt_data[i,0]-rt_data[i+1,0]
dpath[-1] = dpath[-2]

# Compute slant column density of O2 above each atltitude
scdo2 = np.empty(np.size(rt_data[:,0]))
for lev in range(np.size(rt_data[:,0])):
    scdo2[lev] = np.inner(rt_data[0:lev+1,1],dpath[0:lev+1]*1.e5) #km to cm
    
# Compute optical depth for each altitude
tau_O2_l = scdo2*x_o2_l
tau_O2_s = scdo2*x_o2_s

#Compute top-of-atmosphere radiance for LBHl and LBHs
scblbhl = 1e-6*frac_lbhl*np.inner(rt_data[:,2],np.multiply(np.exp(-tau_O2_l),dpath*1e5)) 
scblbhs = 1e-6*frac_lbhs*np.inner(np.multiply(rt_data[:,2],dpath*1e5),np.exp(-tau_O2_s))

print(scblbhl)
print(scblbhs)
        

550.7946840644107
637.3315792662837
